In [1]:
pip install pandas numpy scikit-learn matplotlib nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import string
import tkinter as tk
from tkinter import simpledialog, messagebox
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", "", text)
        return text
    return ""

In [3]:
df = pd.read_csv("review.csv")
df.columns = df.columns.str.strip().str.lower()

if "review" not in df.columns or "sentiment" not in df.columns:
    raise KeyError("Dataset must contain 'review' and 'sentiment' columns")

df["review"] = df["review"].apply(preprocess_text)


In [4]:
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative": 0})

In [11]:
df = df.dropna()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["sentiment"], test_size=0.2, random_state=42)


In [7]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [8]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [9]:
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))

def predict_sentiment(review):
    review = preprocess_text(review)
    review_tfidf = vectorizer.transform([review])
    prediction = model.predict(review_tfidf)[0]
    confidence = model.predict_proba(review_tfidf).max()
    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment, confidence

def generate_suggestion(review):
    keywords = {
        "dress": "The dress has a beautiful design and comfortable fabric. A wider variety of colors would be a great addition!",
        "helper": "The helper was very courteous and efficient. A little more attention to detail would make the service perfect.",
        "electrician": "The electrician did a professional job and fixed the issue quickly. Offering a maintenance checklist might improve customer experience.",
        "product": "The product has great potential and performs well in most aspects. A few improvements in durability would make it even better."
    }
    for key, suggestion in keywords.items():
        if key in review.lower():
            return suggestion
    return "Your feedback is valuable! Constructive suggestions can help improve the experience."


Accuracy: 0.8271604938271605


In [10]:
root = tk.Tk()
root.withdraw()  # Hide the main window

new_review = simpledialog.askstring("Input", "Enter a review:")
if new_review:
    sentiment, confidence = predict_sentiment(new_review)
    suggestion = generate_suggestion(new_review)
    messagebox.showinfo("Result", f"Review: {new_review}\nSentiment: {sentiment} ({confidence*100:.2f}% confidence)")
    messagebox.showinfo("Suggestion", f"Suggested positive feedback: \n\n{suggestion}")